!pip install python-Levenshtein
!git clone --recursive https://github.com/parlance/ctcdecode.git
!pip install wget
%cd ctcdecode
!pip install .
%cd ..

In [2]:
!pip install tokenizers

     |████████████████████████████████| 1.1 MB 3.9 MB/s eta 0:00:01


In [120]:
import numpy as np
import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.utils.rnn import * # for pad_sequence and whatnot
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split

from torch.utils import data
from torchvision import transforms

import matplotlib.pyplot as plt
import time

import json

from transformers import AutoModel, AutoTokenizer, BertTokenizer

"""
from tokenizers import Tokenizer
from tokenizers.decoders import ByteLevel as ByteLevelDecoder
from tokenizers.models import BPE 
from tokenizers.normalizers import Lowercase, NFKC, Sequence
from tokenizers.pre_tokenizers import ByteLevel
"""
cuda = torch.cuda.is_available()
cuda

False

In [37]:
# To have more information about what's happening under the hood
import logging
logging.basicConfig(level=logging.INFO)

!git clone --recursive https://github.com/parlance/ctcdecode.git
!pip install wget
%cd ctcdecode
!pip install .
%cd ..

### DATA PROCESSING

In [41]:
torch.set_grad_enabled(False)
MODEL_NAME = "bert-base-uncased"

model = AutoModel.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /Users/manuelladron/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685
INFO:transformers.configuration_utils:Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
 

In [123]:
ALPHABET = 'abcdefghijklmnopqrstuvwxyz'
NUMBERS = '0123456789'
class PreprocessedData_wordlevel(object):
    def __init__(self, train_file_path, test_file_path):
        """
        train_file_path = list with files
        test_file_path = list with files
        """
        
        self.train_path = train_file_path
        self.test_path = test_file_path
        
        self.VOCAB = None
        self.VOCAB_SIZE = None
                
        # Automatically run it when making an instance
        self.RUN_for_dataset()

    ############ utils #########################
    
    def get_file(self, path):
        with open(path, encoding='utf-8') as f:
            data = json.loads(json.load(f))
        return data
    
    def text_from_json(self, json_file):
        all_text = []
        for file in json_file:
            for sample in file:
                text_l = sample['text']
                for sentence in text_l:
                    sent = sentence.lower()
                    all_text.append(sent)
        return all_text
    

    ############## PROCESSING DATA ##############
    
    def remove_all_letters_in_text_tags_from_alphabet(self, alphabet, positive_tags, all_text):
        """
        Takes in an alphabet, a list of tags and a list of sentences. Returns an alphabet that correspond to the 
        negative samples by substracting the positive tags
        """
        # 1) Get alphabet cropped to the length of sentences
        idx_len = len(all_text)
        cropped_alphabet = alphabet[:idx_len]


        # 2) If not positive tags, return cropped_alpha
        if positive_tags == []:
            return cropped_alphabet

        # 3) Iterate over positive tags and remove them from cropped_alphabet. 
        for tag in positive_tags:
            new_alphabet = cropped_alphabet.replace(tag, "")
            cropped_alphabet = new_alphabet

        # 4) The result is the negative tags! :)
        return new_alphabet

        
    def get_all_text(self, files):
        """
        Parse json file and outputs train_data (text) and numpy array labels for binary classification
        """
        self.sentences = []
        self.sentences_labels = []
        
        for file in files:
            # iterate over the examples in file and grab positive and negative samples
            for i in range(len(file)):
                # elements from dictionary
                positive_tags = file[i]['text-tags']
                text_list = file[i]['text']

                # valid text
                valid_text = [ text_list[ALPHABET.index(letter)].lower() for letter in positive_tags ]

                # nonvalid text
                negative_tags = self.remove_all_letters_in_text_tags_from_alphabet(ALPHABET, positive_tags, text_list)
                nonvalid_text = [ text_list[ALPHABET.index(letter)].lower() for letter in negative_tags ] 
                
                # labels
                pos_label = np.array([0,1])
                neg_label = np.array([1,0])

                # append sentences and labels that are not empty lists
                if len(nonvalid_text) != 0:
                    
                    for nv_text in nonvalid_text:
                        self.sentences.append(nv_text)
                        self.sentences_labels.append(neg_label)

                if len(valid_text) != 0:
                    
                    for v_text in valid_text:
                        self.sentences.append(v_text)
                        self.sentences_labels.append(pos_label)

        # from list to tensor
        self.sentences_labels = np.array(self.sentences_labels)
        self.sentences_labels = torch.from_numpy(self.sentences_labels)
        
    def tokenize_sentences(self):
        
        # Tokenize all sentences and map the tokens to their word ID 
        self.inputs_ids = []
        self.attention_masks = []
        
        for sent in self.sentences:                
            # `encode_plus` will:
            #   (1) Tokenize the sentence.
            #   (2) Prepend the `[CLS]` token to the start.
            #   (3) Append the `[SEP]` token to the end.
            #   (4) Map tokens to their IDs.
            #   (5) Pad or truncate the sentence to `max_length`
            #   (6) Create attention masks for [PAD] tokens.
            encoded_dict = tokenizer.encode_plus(sent,
                                                add_special_tokens = True, # add [CLS] and [SEP]
                                                max_length = 64,
                                                pad_to_max_length = True,  # pad and truncate
                                                return_attention_mask = True,
                                                return_tensors = 'pt'
                                                )
            self.inputs_ids.append(encoded_dict['input_ids'])
            self.attention_masks.append(encoded_dict['attention_mask'])
            
#             print('original: ', sent)
#             print('token IDs: ', encoded_dict['input_ids'])

        # Convert list to tensors
        self.inputs_ids = torch.cat(self.inputs_ids, dim=0)
        self.attention_masks = torch.cat(self.attention_masks, dim=0)
        
    def partition_data(self, train_percentage):
        
        assert len(self.inputs_ids) == len(self.sentences_labels)
        dataset = TensorDataset(self.inputs_ids, self.attention_masks, self.sentences_labels)
        
        train_size = int(train_percentage * len(dataset))
        dev_size = len(dataset) - train_size

        
        self.train_dataset, self.dev_dataset = random_split(dataset, [train_size, dev_size])
        
        print('{:>5,} training samples'.format(train_size))
        print('{:>5,} validation samples'.format(dev_size))

    
    def RUN_for_dataset(self):
        
        # 1) get jsons
        train_raw = []
        for i in range(len(self.train_path)): # list with all training data from different sections
            train_raw.append(self.get_file(self.train_path[i]))
        
        # 2) get text
        self.get_all_text(train_raw)
        print(len(self.sentences), len(self.sentences_labels))
        # 3) tokenize at word-level
        self.tokenize_sentences()
        
        # 4) partition data
        self.partition_data(.8)

In [124]:
dataset = PreprocessedData_wordlevel(["./data/architecture_dz-cleaned-tagged.json",
                            "./data/design_dz-cleaned-tagged.json",
                           "./data/technology_dz-cleaned-tagged.json", ], 
                           ["./data/architecture_dz-cleaned.json", 
                            "./data/design_dz-cleaned.json",
                           "./data/technology_dz-cleaned.json"])

9628 9628
7,702 training samples
1,926 validation samples
